
## Superstore Sales Prediction  
  
Given *data about sales at a superstore*, let's try to predict the **revenue generated** from a given sale.  
  
We will use a TensorFlow neural network to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from sklearn.metrics import r2_score

In [ ]:
data = pd.read_csv('../input/sales-forecasting/train.csv')

In [ ]:
data

In [ ]:
data.info()

# Preprocessing

In [ ]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop unnecessary columns
    df = df.drop(['Row ID', 'Customer Name', 'Country', 'Product Name'], axis=1)
    
    # Drop customer-specific feature columns
    df = df.drop(['Order ID', 'Customer ID'], axis=1)
    
    # Extract date features
    df = encode_dates(df, column='Order Date')
    df = encode_dates(df, column='Ship Date')
    
    # One-hot encode categorical features
    for column in ['Ship Mode', 'Segment', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category']:
        df = onehot_encode(df, column=column)
    
    # Split df into X and y
    y = df['Sales']
    X = df.drop('Sales', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train

In [ ]:
y_train

# Training

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

# Results

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))

In [ ]:
y_pred = np.squeeze(model.predict(X_test))
test_r2 = r2_score(y_test, y_pred)

print("Test R^2 Score: {:.5f}".format(test_r2))